In [11]:
pip install opencv-python --upgrade


In [25]:
import cv2
import pytesseract
from openpyxl import load_workbook, Workbook
import re

# Load Tesseract and set configurations
pytesseract.pytesseract.tesseract_cmd = r"C:\\Program Files\\Tesseract-OCR\\tesseract.exe"
custom_config = r'--oem 3 --psm 6'

# Function to extract text from image using pytesseract
def extract_text_from_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray, config=custom_config)
    return text

# Function to extract certificate details from text
def extract_certificate_details(text):
    name_pattern = re.compile(r'Name (.+?)\n')
    name_match = name_pattern.search(text)
    name_cert = name_match.group(1) if name_match else None

    ins_pattern = re.compile(r'Institution (.+?)\n')
    ins_match = ins_pattern.search(text)
    ins_cert = ins_match.group(1) if ins_match else None

    unit_pattern = re.compile(r'by (.+?)\n')
    unit_match = unit_pattern.search(text)
    unit_cert = unit_match.group(1) if unit_match else None

    pl_pattern = re.compile(r'Place : (\w+)')
    pl_match = pl_pattern.search(text)
    pl_cert = pl_match.group(1) if pl_match else None
    
    dt_pattern = re.compile(r'Date: (\d{1,2} \w+ \d{4})')
    dt_match = dt_pattern.search(text)
    dt_cert = dt_match.group(1) if dt_match else None

    rank_pattern = re.compile(r'Rank, (.+?)\n')
    rank_match = rank_pattern.search(text)
    rank_cert = rank_match.group(1) if rank_match else None

    return name_cert, ins_cert, unit_cert, pl_cert, dt_cert, rank_cert

# Load the existing workbook or create a new one if it doesn't exist
try:
    wb = load_workbook('Nanddy0031.xlsx')
except FileNotFoundError:
    wb = Workbook()
    ws = wb.active
    ws.append(['Name', 'Institution', 'By', 'Place', 'Date', 'Rank'])

ws = wb.active

import cv2
import urllib.request
import numpy as np

# Replace 'your_ip_address:port' with your Android device's IP address and port
# Initialize webcam
cap = cv2.VideoCapture('http://192.168.36.229:8080/video')

while True:
    ret, frame = cap.read()
    if not ret:
        break

    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


text = extract_text_from_image(frame)
name_cert, ins_cert, unit_cert, pl_cert, dt_cert, rank_cert = extract_certificate_details(text)
ws.append([name_cert, ins_cert, unit_cert, pl_cert, dt_cert, rank_cert])

# Save the workbook
wb.save('Nanddy0031.xlsx')

# Release the webcam and close OpenCV windows
cap.release()
cv2.destroyAllWindows()